# Create Sequences for Dataset

In [1]:
import pandas as pd
import numpy as np
import copy
import gumpy

### Get sequences from mutations

In [2]:
reference = gumpy.Genome('../data/NC_000962.3.gbk')
pnca = reference.build_gene('pncA')

In [ ]:
# from fowler-lab/predict-pyrazinamide-resistance GitHub

pnca_paper_dataset = pd.read_csv('../data/ds-traintest-phen.csv')

In [4]:
def split_mutation(row):
    return pd.Series([row.MUTATION[0], int(row.MUTATION[1:-1]), row.MUTATION[-1]])

pnca_paper_dataset[['REF','AMINO_ACID','ALT']] = pnca_paper_dataset.apply(split_mutation,axis=1)

pnca_paper_dataset.head(10)

,MUTATION,CONSISTENT_PHENOTYPE,REF,AMINO_ACID,ALT
0,A102V,S,A,102,V
1,A134D,S,A,134,D
2,A134P,R,A,134,P
3,A134S,S,A,134,S
4,A134V,R,A,134,V
5,A143D,R,A,143,D
6,A143G,R,A,143,G
7,A143V,S,A,143,V
8,A146E,R,A,146,E
9,A146S,R,A,146,S


In [5]:
# create table of sample sequences

sequences = pd.DataFrame(columns=['phenotype_label','number_resistant_mutations', 'number_susceptible_mutations', 'allele', 'mutation'])

for idx, row in pnca_paper_dataset.iterrows():
    
    sample = copy.deepcopy(pnca)
    # reassign amino acid to mutated amino acid
    sample.amino_acid_sequence[sample.amino_acid_number==row.AMINO_ACID] = row.ALT
    
    sequence = ''.join(i for i in sample.amino_acid_sequence)
    mutation = row.MUTATION
    
    if row.CONSISTENT_PHENOTYPE == 'R':
        number_resistant_mutations = 1
        number_susceptible_mutations = 0
        phenotype_label = 'R'
    else:
        number_resistant_mutations = 0
        number_susceptible_mutations = 1
        phenotype_label = 'S'
        
    sequences.loc[len(sequences)] = [phenotype_label, number_resistant_mutations, number_susceptible_mutations, sequence, mutation]
    

In [6]:
display(sequences)
display(sequences.phenotype_label.value_counts())

,phenotype_label,number_resistant_mutations,number_susceptible_mutations,allele,mutation
0,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A102V
1,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134D
2,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134P
3,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134S
4,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134V
...,...,...,...,...,...
659,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y95N
660,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y99C
661,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y99D
662,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y99F


phenotype_label
R    349
S    315
Name: count, dtype: int64

### Get original Train / Test Split

In [ ]:
# from fowler-lab/predict-pyrazinamide-resistance GitHub

with open('../data/ds-train.npy', 'rb') as f:
    y_train = np.load(f)
    x_train = np.load(f)
    train_muts = np.load(f, allow_pickle=True)
    
with open('../data/ds-test.npy', 'rb') as f:
    y_test = np.load(f)
    x_test = np.load(f)
    test_muts = np.load(f, allow_pickle=True)

In [8]:
train_muts_df = pd.DataFrame({'mutation': list(train_muts)})
test_muts_df = pd.DataFrame({'mutation': list(test_muts)})

train_seqs = pd.merge(sequences, train_muts_df, on='mutation', how='inner')
test_seqs = pd.merge(sequences, test_muts_df, on='mutation', how='inner')   

display(train_seqs)
display(test_seqs)

,phenotype_label,number_resistant_mutations,number_susceptible_mutations,allele,mutation
0,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A102V
1,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134D
2,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134P
3,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134S
4,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A134V
...,...,...,...,...,...
459,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDHLAEAADYHHVVA...,Y34H
460,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y64N
461,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y95H
462,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y99C


,phenotype_label,number_resistant_mutations,number_susceptible_mutations,allele,mutation
0,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A143D
1,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A143V
2,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A146E
3,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A161T
4,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,A178D
...,...,...,...,...,...
195,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADHHHVVA...,Y41H
196,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y95C
197,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y95N
198,S,0,1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,Y99D


In [9]:
with open('../data/real_train_sequences.csv', 'wb') as file:
    train_seqs.to_csv(file, index=False)

with open('../data/real_test_sequences.csv', 'wb') as file:
    test_seqs.to_csv(file, index=False)

### Create FASTA files for AlphaFold

In [10]:
def save_fasta(header, sequence, name):
    filename = f"../data/fastas/{name}.fasta"
    """Save a sequence in FASTA format to a file."""
    with open(filename, "w") as fasta_file:
        fasta_file.write(f">{header}\n")
        fasta_file.write(sequence)

In [11]:
filename = "../data/fastas/train.fasta"

with open(filename, 'w') as fasta_file:
    
    for index, seq_row in train_seqs.iterrows():
        res_index = int(seq_row.mutation[1:-1]) -1
        alt_res = seq_row.mutation[-1]
        assert alt_res == seq_row.allele[res_index], 'Mutation does not match sequence \n' + str(seq_row.mutation) + '\n' + str(seq_row.allele)

        sequence = seq_row.allele
        
        fasta_file.write(f">{index}\n")
        fasta_file.write(f"{sequence}\n")
        

In [12]:
filename = "../data/fastas/test.fasta"

with open(filename, 'w') as fasta_file:
    
    for index, seq_row in test_seqs.iterrows():
        res_index = int(seq_row.mutation[1:-1]) -1
        alt_res = seq_row.mutation[-1]
        assert alt_res == seq_row.allele[res_index], 'Mutation does not match sequence \n' + str(seq_row.mutation) + '\n' + str(seq_row.allele)

        sequence = seq_row.allele
        
        fasta_file.write(f">{index}\n")
        fasta_file.write(f"{sequence}\n")